In [1]:
from libs.AirCraft import *

In [2]:
orbit1_alt = 300 # km 
orbit2_alt = 500 # km 

sat1_1 = Satellite()

In [3]:
import math

def get_points_on_circle(d: float, num_points: int) -> list:
    points = []
    theta_gap = 2 * math.pi / num_points  # 일정한 간격
    for i in range(num_points):
        theta = i * theta_gap
        x = d * math.cos(theta)
        y = d * math.sin(theta)
        points.append((x, y))
    return points

points = get_points_on_circle(5, 8)
print(points)


[(5.0, 0.0), (3.5355339059327378, 3.5355339059327378), (3.061616997868383e-16, 5.0), (-3.5355339059327373, 3.5355339059327378), (-5.0, 6.123233995736766e-16), (-3.5355339059327386, -3.5355339059327373), (-9.184850993605148e-16, -5.0), (3.535533905932737, -3.5355339059327386)]
